## 查询组件

这个查询组件会根据用户的输入，首先提取输入的关键字，然后用google search查询。最后生成一段解释的文本返回。

In [ ]:
from typing import Any, Dict, List, Optional
from langchain.prompts import PromptTemplate
from langchain.chains.base import Chain
from langchain import BasePromptTemplate
from langchain.schema.language_model import BaseLanguageModel
from langchain.schema import LLMResult
from langchain.chains import LLMChain
from pydantic import Extra, Field
from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
)
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.memory import ConversationBufferMemory

import json
from datetime import datetime
from DateConvert import Lunar
from langchain.utilities import GoogleSerperAPIWrapper


class SearchChain(Chain):
    """
    An example of a custom chain.
    """

    prompt: BasePromptTemplate
    res_prompt: BasePromptTemplate
    """Prompt object to use."""
    llm: BaseLanguageModel
    output_key: str = "text"  #: :meta private:

    search: GoogleSerperAPIWrapper

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the prompt expects.

        :meta private:
        """
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """Will always return text key.

        :meta private:
        """
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that mimics LLMChain
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
            run_manager.on_text(
                prompt_value.to_string(), color="green", end="\n", verbose=self.verbose
            )
        res = self.llm.generate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )
        terms = res.generations[0][0].text
        result = self.search.run(terms)
        prompt_value = self.res_prompt.format_prompt(
            question=inputs["question"], result=result
        )
        if run_manager:
            run_manager.on_text(
                prompt_value.to_string(), color="green", end="\n", verbose=self.verbose
            )
        res = self.llm.generate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )
        return {self.output_key: res.generations[0][0].text}

    async def _acall(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that mimics LLMChain
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
            await run_manager.on_text(
                prompt_value.to_string(), color="green", end="\n", verbose=self.verbose
            )
        res = await self.llm.agenerate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )
        terms = res.generations[0][0].text
        result = await self.search.arun(terms)
        prompt_value = self.res_prompt.format_prompt(
            question=inputs["question"], result=result
        )
        if run_manager:
            await run_manager.on_text(
                prompt_value.to_string(), color="green", end="\n", verbose=self.verbose
            )
        res = await self.llm.agenerate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )
        return {self.output_key: res.generations[0][0].text}

    @property
    def _chain_type(self) -> str:
        return "calendar_chain"

    @classmethod
    def from_llm(
        cls,
        llm: BaseLanguageModel,
        **kwargs: Any,
    ) -> Chain:
        template = """说明
---------------------
- 下面的问题是人类向你发出的提问
- 如果你需要搜索互联网，请把关键词提取出来，作为search term

问题
---------------------
{question}

Search Term
---------------------
"""
        res_template = """说明
---------------------
- 请根据搜索结果，解释下面问题里的关键词

搜索结果
---------------------
{result}

问题
---------------------
{question}

解释
---------------------
"""
        prompt = PromptTemplate.from_template(template=template)
        res_prompt = PromptTemplate.from_template(template=res_template)
        search = GoogleSerperAPIWrapper()
        return cls(
            llm=llm, prompt=prompt, res_prompt=res_prompt, search=search, **kwargs
        )

## 一个用Chain实现的chatbot

后面demo中解释如何使用搜索组件时需要使用。

In [ ]:
from langchain.llms import OpenAI


class StarLoomChain(LLMChain):
    """
    An example of a custom chain.
    """

    prompt: BasePromptTemplate
    """Prompt object to use."""
    llm: BaseLanguageModel
    output_key: str = "text"  #: :meta private:

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the prompt expects.

        :meta private:
        """
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """Will always return text key.

        :meta private:
        """
        return [self.output_key]

    def generate(
        self,
        input_list: List[Dict[str, Any]],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> LLMResult:
        prompts, stop = self.prep_prompts(input_list, run_manager=run_manager)
        return self.llm.generate_prompt(
            prompts=prompts,
            stop=stop,
            callbacks=run_manager.get_child() if run_manager else None,
        )

    async def agenerate(
        self,
        input_list: List[Dict[str, Any]],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> LLMResult:
        prompts, stop = await self.aprep_prompts(input_list, run_manager=run_manager)
        return await self.llm.agenerate_prompt(
            prompts=prompts,
            stop=stop,
            callbacks=run_manager.get_child() if run_manager else None,
        )

    @property
    def _chain_type(self) -> str:
        return "starloom_chain"

    @classmethod
    def from_llm(
        cls,
        llm: BaseLanguageModel,
        **kwargs: Any,
    ) -> Chain:
        prompt = ChatPromptTemplate(
            messages=[
                SystemMessagePromptTemplate.from_template(
                    """说明
------------------
- 你的名字叫伏羲，是一个精通周易、中国生肖、算命、解梦、风水堪舆、西方塔罗牌和占星术的玄学大师。正在与人类进行对话。
- 你精通各种玄学和命理知识，能够帮助人类提供命理方面的解读和建议。你在与人类交谈时，常常会点燃你的烟斗。你在烟草的烟气中思考人类提出的问题。
- 你本身具有很神秘的性格，外表看起来是一个睿智的老者。
- 你一直用占卜师带有神秘感的口吻与人类交谈。

卜卦的方法
------------------
周易卦象是一种古老的预测方法。如果你有硬币，我们可以试一试。这需要你准备三枚相同的硬币，然后抛掷六次，记录每次抛掷的结果。

- 每次抛掷后，正面（通常是带头像的一面）记为3，反面（通常是带数字的那一面）记为2。
- 每次抛掷三个硬币，把每次抛掷的结果相加，你会得到一个6，7，8或9的数字。
- 6或8的数字画成阴爻（断的线），7或9的数字画成阳爻（一条完整的线）。
- 第一次抛掷的结果放在最下面，然后依次向上叠加画出卦象，形成一个由六个爻组成的卦象。

注意
------------------
- 人类提到了日期时，如果没有特别说明是农历（阴历）日期，那么请你默认为公历日期。
"""
                ),
                MessagesPlaceholder(variable_name="chat_history"),
                HumanMessagePromptTemplate.from_template("{question}"),
            ]
        )
        memory = ConversationBufferMemory(
            memory_key="chat_history", return_messages=True
        )

        return cls(
            llm=llm,
            prompt=prompt,
            memory=memory,
            **kwargs,
        )


## Demo，如何在chatbot中使用搜索

下面是初始chatbot

In [ ]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv(dotenv_path="env")

gpt35 = ChatOpenAI(temperature=0.9, verbose=True)
gpt4 = ChatOpenAI(model_name="gpt-4", temperature=0.9, verbose=True)
starloom_chain = StarLoomChain.from_llm(
    llm=gpt4,
    verbose=True,
)

下面是初始化搜索组件

In [ ]:
gpt35_1 = OpenAI(temperature=0.1, verbose=True)
search_chain = SearchChain.from_llm(llm=gpt35_1, verbose=True)

下面是启动chatbot。
用户在输入提问后，通过搜索组件处理用户的提问。获得搜索结果`search_result`
```python
search_result = await search_chain.arun(user_input)
```

然后，把搜索结果和用户输入，通过一个模板，整合成一个新的提问字符串，让chatbot来回答。

模板
```python
user_input_template = """查询系统提示
------------------
- {search_result}

{question}"""
```

`search_result`就是搜索结果

`question`就是用户输入的提问

In [ ]:
while True:
    user_input = input("Your prompt: ")
    if user_input == ":exit":
        break
    search_result = await search_chain.arun(user_input)
    user_input_template = """查询系统提示
------------------
- {search_result}

{question}"""
    question = PromptTemplate.from_template(template=user_input_template).format_prompt(
        question=user_input, search_result=search_result
    ).to_string()
    res = await starloom_chain.arun(question=question)
    print(res)